In [1]:
import os
import numpy as np
import pandas as pd

os.chdir("/Users/cornederuijt/github/GCM/") # Adjust after construction of the package

from scripts.clickmodel_fitters.clickdefinitionreader import ClickDefinition
from scripts.clickmodel_fitters.GCM import GCM

import tensorflow as tf
import tensorflow_probability as tfp
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import Layer

np.random.seed(1992)

In [2]:
class SimpleDense(Layer):
    def __init__(self, units=11):
        super(SimpleDense, self).__init__()
        self.units = units

    def build(self, input_shape):  # Create the state of the layer (weights)
        T = self.units - 1  # As we exclude the zero state for now
        zero_init = tf.zeros_initializer()

        self._w = tf.Variable(initial_value=zero_init(shape=(int(T * (T + 1) / 2),), dtype='float32'), trainable=True)
        self._zero_col_var = tf.Variable(initial_value=zero_init(shape=(T + 1, 1), dtype='float32'), trainable=False)
        self._zero_row_var = tf.Variable(initial_value=zero_init(shape=(1, T), dtype='float32'), trainable=False)

        # mask:
        t1 = tf.ones((T+1, T+1), dtype="float32")
        t2 = tf.subtract(t1, tf.eye(T+1, dtype="float32"))
        self._wout_mask = tf.multiply(tf.linalg.band_part(t1, 0, -1), t2)

    def call(self, inputs, **kwargs):  # Defines the computation from inputs to outputs
        # print(tf.shape(inputs)[0])
        N = inputs.shape[0]
        T = self.units
        inp_lst = []

        wout = tfp.math.fill_triangular(self._w, upper=True)
        wout = tf.concat([wout, self._zero_row_var], axis=0)
        wout = tf.concat([self._zero_col_var, wout], axis=1)

        for t in range(N):
            cur_input = tf.gather(inputs, t, axis=0)
            cur_input = tf.tile(tf.reshape(cur_input, shape=(1, -1)), [T, 1])
            logis = tf.multiply(cur_input, wout)

            cur_mask = tf.multiply(cur_input, self._wout_mask)

            inp_lst.append(tf.reshape(
                tf.multiply(tf.transpose(tf.transpose(tf.exp(logis)) / tf.reduce_sum(tf.exp(logis), axis=1)),
                              cur_mask), shape=(1, -1)))

        res = tf.concat(inp_lst, axis=0)

        return res

In [3]:
# Define the model
list_size = 10
no_states = 11
ubm_click_states = np.eye(list_size + 1, list_size + 1)
abs_state = [(i, i) for i in range(no_states)]
init_state = 0
batch_size = 10000
no_items = 100

var_dic = {
    'phi_A': {
        'var_type': 'item',
        'pos_mat': np.triu(np.ones((list_size + 1, list_size + 1)), k=1)
    },
    'gamma': {
        'var_type': 'pos',
        'pos_mat': np.triu(np.ones((list_size + 1, list_size + 1)), k=1)
    }
}

model_def = ClickDefinition(ubm_click_states, init_state, list_size, no_states, batch_size, no_items, abs_state,
                            var_dic)

In [4]:
# Load data:
click_data = pd.read_csv("./data/small_example/simulation_res_train.csv", index_col=False)
prod_position = pd.read_csv("./data/small_example/simulation_item_props.csv", index_col=False)

# Ensure the order is correct:
click_data = click_data.sort_values(['user_id', 'session_count', 'item_order'])

# Add session index:
session_index = (click_data
                 .loc[:, ['user_id', 'session_count']]
                 .drop_duplicates()
                 .reset_index()
                 )

session_index['session'] = session_index.index.to_numpy()

click_data = (click_data
              .set_index(['user_id', 'session_count'])
              .join(session_index
                    .set_index(['user_id', 'session_count']),
                    on=['user_id', 'session_count'])
              .reset_index()
              .set_index('item')
              .join(prod_position
                    .set_index('item'),
                    on='item')
              .reset_index()
              )

click_data



,item,user_id,session_count,item_order,click,attr,satis,eval,orig_list_id,index,session,X0,X1
0,42,0,0,1,0.0,0,0,1.0,0,0,0,-0.617880,-0.424150
1,66,0,0,2,0.0,0,0,1.0,0,0,0,-0.463804,-0.667938
2,89,0,0,3,0.0,0,0,1.0,0,0,0,-0.849402,1.813430
3,64,0,0,4,0.0,0,0,1.0,0,0,0,-1.875517,-1.026840
4,25,0,0,5,0.0,0,0,1.0,0,0,0,-0.908807,0.050236
...,...,...,...,...,...,...,...,...,...,...,...,...,...
250565,85,17999,2,6,0.0,0,0,0.0,35807,250560,25056,-0.362179,-0.737943
250566,3,17999,2,7,0.0,1,0,0.0,35807,250560,25056,0.136692,0.039374
250567,76,17999,2,8,0.0,1,0,0.0,35807,250560,25056,0.257214,-0.653226
250568,64,17999,2,9,0.0,0,0,0.0,35807,250560,25056,-1.875517,-1.026840


In [5]:
# Create the click matrix and item position matrix
click_mat = click_data.loc[:, ['session', 'item_order', 'click']] \
    .pivot(index='session', columns='item_order', values='click') \
    .to_numpy()

item_pos_mat = click_data.loc[:, ['session', 'item_order', 'item']] \
    .pivot(index='session', columns='item_order', values='item') \
    .to_numpy()

# Ensure the order is correct
item_feature_mat_A = (click_data.loc[:, ['item', 'X0', 'X1']]
                                .drop_duplicates()
                                .sort_values('item')
                                .to_numpy())

pos_feature_gamma = np.eye(model_def.list_size, model_def.list_size + 1, k=1)

var_dic = {'phi_A': item_feature_mat_A, 'gamma': pos_feature_gamma}

In [6]:
# Model:
model_phi_A = Sequential()
model_phi_A.add(Dense(1, input_dim=var_dic['phi_A'].shape[1], activation='sigmoid', use_bias=False))
model_phi_A.compile(loss=GCM.pos_log_loss, optimizer=RMSprop())

# Note the large output dimension and the softmax. We want multiple transition probabilities that sum up to 1
# Its the shape**2, as we flatten the square matrix.
model_gamma = Sequential()
model_gamma.add(SimpleDense(no_states))
model_gamma.compile(loss=GCM.pos_log_loss, optimizer=RMSprop())

# model_tau = Sequential()
# model_tau.add(Dense(var_dic['tau'].shape[1], input_dim=var_dic['tau'].shape[1], activation=None, use_bias=False,
#                     kernel_initializer=Identity(), trainable=False))
# model_tau.compile('rmsprop', 'binary_crossentropy')  # No trainable weights, so doesn't really matter

var_models = {'phi_A': model_phi_A, 'gamma': model_gamma}

In [7]:
res = GCM.runEM(click_mat, var_dic, var_models, item_pos_mat, model_def, verbose=True)

Iteration: 0
Running E-step ...
Current conditional entropy:0.57217
Running M-step ...
Epoch 1/250
1/1 [==============================] - 0s 1ms/step - loss: 53481.0039
Epoch 2/250
1/1 [==============================] - 0s 522us/step - loss: 49215.1094
Epoch 3/250
1/1 [==============================] - 0s 550us/step - loss: 46284.0312
Epoch 4/250
1/1 [==============================] - 0s 748us/step - loss: 43935.1094
Epoch 5/250
1/1 [==============================] - 0s 518us/step - loss: 41931.4336
Epoch 6/250
1/1 [==============================] - 0s 752us/step - loss: 40163.4102
Epoch 7/250
1/1 [==============================] - 0s 469us/step - loss: 38570.0469
Epoch 8/250
1/1 [==============================] - 0s 714us/step - loss: 37113.3945
Epoch 9/250
1/1 [==============================] - 0s 468us/step - loss: 35767.9844
Epoch 10/250
1/1 [==============================] - 0s 650us/step - loss: 34515.7969
Epoch 11/250
1/1 [==============================] - 0s 447us/step - loss: 

Epoch 97/250
1/1 [==============================] - 0s 694us/step - loss: 6280.2280
Epoch 98/250
1/1 [==============================] - 0s 695us/step - loss: 6209.6719
Epoch 99/250
1/1 [==============================] - 0s 548us/step - loss: 6140.4805
Epoch 100/250
1/1 [==============================] - 0s 848us/step - loss: 6072.6123
Epoch 101/250
1/1 [==============================] - 0s 4ms/step - loss: 6006.0298
Epoch 102/250
1/1 [==============================] - 0s 730us/step - loss: 5940.6973
Epoch 103/250
1/1 [==============================] - 0s 654us/step - loss: 5876.5796
Epoch 104/250
1/1 [==============================] - 0s 1ms/step - loss: 5813.6411
Epoch 105/250
1/1 [==============================] - 0s 529us/step - loss: 5751.8511
Epoch 106/250
1/1 [==============================] - 0s 777us/step - loss: 5691.1777
Epoch 107/250
1/1 [==============================] - 0s 2ms/step - loss: 5631.5894
Epoch 108/250
1/1 [==============================] - 0s 713us/step - loss:

1/1 [==============================] - 0s 995us/step - loss: 2773.3408
Epoch 194/250
1/1 [==============================] - 0s 625us/step - loss: 2755.0588
Epoch 195/250
1/1 [==============================] - ETA: 0s - loss: 2736.97 - 0s 1ms/step - loss: 2736.9751
Epoch 196/250
1/1 [==============================] - 0s 819us/step - loss: 2719.0862
Epoch 197/250
1/1 [==============================] - 0s 693us/step - loss: 2701.3906
Epoch 198/250
1/1 [==============================] - 0s 1ms/step - loss: 2683.8838
Epoch 199/250
1/1 [==============================] - 0s 709us/step - loss: 2666.5637
Epoch 200/250
1/1 [==============================] - 0s 652us/step - loss: 2649.4272
Epoch 201/250
1/1 [==============================] - 0s 2ms/step - loss: 2632.4724
Epoch 202/250
1/1 [==============================] - 0s 569us/step - loss: 2615.6960
Epoch 203/250
1/1 [==============================] - 0s 3ms/step - loss: 2599.0967
Epoch 204/250
1/1 [==============================] - 0s 742us

1/1 [==============================] - 0s 531us/step - loss: 48273488.0000
Epoch 34/250
1/1 [==============================] - 0s 612us/step - loss: 48273384.0000
Epoch 35/250
1/1 [==============================] - 0s 506us/step - loss: 48273340.0000
Epoch 36/250
1/1 [==============================] - 0s 605us/step - loss: 48273248.0000
Epoch 37/250
1/1 [==============================] - 0s 709us/step - loss: 48273176.0000
Epoch 38/250
1/1 [==============================] - 0s 625us/step - loss: 48273080.0000
Epoch 39/250
1/1 [==============================] - 0s 618us/step - loss: 48273000.0000
Epoch 40/250
1/1 [==============================] - 0s 598us/step - loss: 48272960.0000
Epoch 41/250
1/1 [==============================] - 0s 618us/step - loss: 48272840.0000
Epoch 42/250
1/1 [==============================] - 0s 689us/step - loss: 48272768.0000
Epoch 43/250
1/1 [==============================] - 0s 627us/step - loss: 48272712.0000
Epoch 44/250
1/1 [===========================

1/1 [==============================] - 0s 1ms/step - loss: 48266088.0000
Epoch 127/250
1/1 [==============================] - 0s 2ms/step - loss: 48265972.0000
Epoch 128/250
1/1 [==============================] - 0s 613us/step - loss: 48265900.0000
Epoch 129/250
1/1 [==============================] - 0s 908us/step - loss: 48265816.0000
Epoch 130/250
1/1 [==============================] - 0s 1ms/step - loss: 48265776.0000
Epoch 131/250
1/1 [==============================] - 0s 1ms/step - loss: 48265656.0000
Epoch 132/250
1/1 [==============================] - 0s 719us/step - loss: 48265604.0000
Epoch 133/250
1/1 [==============================] - 0s 694us/step - loss: 48265496.0000
Epoch 134/250
1/1 [==============================] - 0s 613us/step - loss: 48265412.0000
Epoch 135/250
1/1 [==============================] - 0s 938us/step - loss: 48265344.0000
Epoch 136/250
1/1 [==============================] - 0s 1ms/step - loss: 48265236.0000
Epoch 137/250
1/1 [==========================

1/1 [==============================] - 0s 733us/step - loss: 48258768.0000
Epoch 219/250
1/1 [==============================] - 0s 629us/step - loss: 48258700.0000
Epoch 220/250
1/1 [==============================] - 0s 1ms/step - loss: 48258624.0000
Epoch 221/250
1/1 [==============================] - 0s 1ms/step - loss: 48258532.0000
Epoch 222/250
1/1 [==============================] - 0s 660us/step - loss: 48258452.0000
Epoch 223/250
1/1 [==============================] - 0s 627us/step - loss: 48258392.0000
Epoch 224/250
1/1 [==============================] - 0s 831us/step - loss: 48258304.0000
Epoch 225/250
1/1 [==============================] - 0s 737us/step - loss: 48258256.0000
Epoch 226/250
1/1 [==============================] - 0s 970us/step - loss: 48258160.0000
Epoch 227/250
1/1 [==============================] - 0s 1ms/step - loss: 48258088.0000
Epoch 228/250
1/1 [==============================] - 0s 2ms/step - loss: 48257980.0000
Epoch 229/250
1/1 [========================

1/1 [==============================] - 0s 588us/step - loss: 1475.1707
Epoch 63/250
1/1 [==============================] - 0s 472us/step - loss: 1468.8892
Epoch 64/250
1/1 [==============================] - 0s 609us/step - loss: 1462.6538
Epoch 65/250
1/1 [==============================] - 0s 518us/step - loss: 1456.4645
Epoch 66/250
1/1 [==============================] - 0s 588us/step - loss: 1450.3204
Epoch 67/250
1/1 [==============================] - 0s 514us/step - loss: 1444.2214
Epoch 68/250
1/1 [==============================] - 0s 594us/step - loss: 1438.1667
Epoch 69/250
1/1 [==============================] - 0s 508us/step - loss: 1432.1562
Epoch 70/250
1/1 [==============================] - 0s 625us/step - loss: 1426.1893
Epoch 71/250
1/1 [==============================] - 0s 564us/step - loss: 1420.2660
Epoch 72/250
1/1 [==============================] - 0s 638us/step - loss: 1414.3857
Epoch 73/250
1/1 [==============================] - 0s 549us/step - loss: 1408.5472
Epoch

1/1 [==============================] - 0s 2ms/step - loss: 1028.3834
Epoch 160/250
1/1 [==============================] - 0s 489us/step - loss: 1025.0532
Epoch 161/250
1/1 [==============================] - 0s 639us/step - loss: 1021.7422
Epoch 162/250
1/1 [==============================] - 0s 488us/step - loss: 1018.4504
Epoch 163/250
1/1 [==============================] - 0s 585us/step - loss: 1015.1775
Epoch 164/250
1/1 [==============================] - 0s 488us/step - loss: 1011.9231
Epoch 165/250
1/1 [==============================] - 0s 564us/step - loss: 1008.6873
Epoch 166/250
1/1 [==============================] - 0s 494us/step - loss: 1005.4703
Epoch 167/250
1/1 [==============================] - 0s 605us/step - loss: 1002.2713
Epoch 168/250
1/1 [==============================] - 0s 511us/step - loss: 999.0908
Epoch 169/250
1/1 [==============================] - 0s 621us/step - loss: 995.9280
Epoch 170/250
1/1 [==============================] - 0s 680us/step - loss: 992.7834

AttributeError: Layer simple_dense has no inbound nodes.